<div class="frontmatter">
    <div style="display: flex; align-items: flex-start; justify-content: space-between;">
        <div>
             <h1 style="font-weight: bold;">Práctica 1: Manejo de datos geoespaciales y análisis preliminar</h1>
            <h2>Autores: David Cantó, Roberto Rodero</h2>
            <h3>Máster en Tecnologías de la Información Geográfica, UAH</h3>
            <h3>Asignatura: Programación Avanzada</h3>
        </div>
</div>

# **Análisis de delitos y criminalidad en áreas urbanas**

## **1. Introducción**

### **Contexto**
La criminalidad en entornos urbanos es un fenómeno complejo influenciado por diversos factores socioeconómicos y espaciales. Comprender la distribución geográfica de los delitos es esencial para el diseño de estrategias de prevención y respuesta por parte de las autoridades. Este trabajo tiene como objetivo analizar los delitos cometidos en los distritos de la ciudad de Madrid, enfocándose en diversas categorías, como la seguridad ciudadana, el número de personas detenidas, los atestados y partes de accidentes, el consumo de alcohol en la vía pública, y las inspecciones y actuaciones en locales de espectáculos públicos y actividades recreativas.

El análisis espacial permitirá identificar patrones y tendencias de criminalidad, contribuyendo a una mejor comprensión del fenómeno y proporcionando información útil para la toma de decisiones de seguridad y políticas públicas.

## **Objetivos**
- Analizar la distribución geográfica de los delitos en los distritos de Madrid y su evolución durante la última década.
- Comprobar si la superficie de zonas verdes de cada distrito se relaciona con una mayor concurrencia de delitos.
- Evaluar la relación entre la densidad de población y la criminalidad en los distritos de Madrid.
- Generar cartografía y gráficos estadísticos que ayuden a comprender este fenómeno.

## **Preguntas clave**
1. ¿Cuáles son los distritos de Madrid con mayor número de delitos?
2. ¿Existe una correlación entre la superficie de zonas verdes y la cantidad de delitos en cada distrito?
3. ¿Las áreas con mayor densidad de población registran una mayor incidencia delictiva?
4. ¿Cuál ha sido la evolución del número y tipo de delitos durante la última década?

## **2. Conjunto de datos**

Para este estudio, se utilizarán datos públicos proporcionados por el Ayuntamiento de Madrid, a través del Portalde Datos Abiertos del Ayuntamiento de Madrid y se pueden consultar en el siguiente enlace:  [https://datos.madrid.es/sites/v/index.jsp?vgnextoid=bffff1d2a9fdb410VgnVCM2000000c205a0aRCRD&vgnextchannel=20d612b9ace9f310VgnVCM100000171f5a0aRCRD](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=bffff1d2a9fdb410VgnVCM2000000c205a0aRCRD&vgnextchannel=20d612b9ace9f310VgnVCM100000171f5a0aRCRD). Los principales conjuntos de datos incluyen:

- Delitos relacionados con la seguridad ciudadana:
  - Relacionados con las personas
  - Relacionadas con el patrimonio
  - Por tenencia de armas
  - Por tenencia de drogas
  - Por consumo de drogas

- Personas detenidas e imputadas:

- Partes de accidente confeccionados y atestados:

- Consumo de alcohol en la vía pública:

- Intervenciones por protección a los consumidores y usuarios en vía pública:

- Inspecciones y actuaciones en locales de espectáculos públicos y actividades recreativas:

- Capa de distritos de la ciudad de Madrid

Se han utilizado datos de población por distritos, disponibles en el siguiente enlace: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=0cccaebc07c1f710VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default.

Estos datos serán analizados en formato geoespacial (SHP, GeoJSON) y tabular (CSV, Excel), considerando su calidad, precisión y confiabilidad para garantizar resultados válidos.

Por su parte, el archivo ráster a analizar es un ráster de zonas verdes de la ciudad de Madrid, disponible en: https://geoportal.madrid.es/IDEAM_WBGEOPORTAL/dataset.iam?id=53896e7f-8b6b-4cbc-a0aa-bfbeee91563e.

## **3. Exploración y procesamiento de datos**

En primer lugar se importaron los módulos y paquetes necesarios para poder explorar y procesar los conjuntos de datos con los que vamos a trabajar.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import folium
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
from rasterstats import zonal_stats
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import branca.colormap as cm
from sklearn.linear_model import LinearRegression
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose
from google.colab import drive
from google.colab import files
import dask.array as da
import mapclassify

Una vez cargados los paquetes necesarios, preparamos la carpeta de google drive desde la que vamos a trabajar. Dentro de ella se encuentran los conjuntos de datos que vamos a utilizar.

In [ ]:
# Conectar a Google Drive para enlazar las carpetas almacenadas en Drive con los archivos necesarios
drive.mount('/content/drive')
# Establecer la ruta de la carpeta principal
folder = '/content/drive/My Drive/P1_Avanzada'

# Listar las subcarpetas dentro de la carpeta principal
def listar_carpetas(carpeta):
    for name in os.listdir(carpeta):
        if os.path.isdir(os.path.join(carpeta, name)):
            print(name)

listar_carpetas(folder)

In [ ]:
# Establecer la ruta que contiene las carpetas con los datos de delitos, distritos y ráster de zonas verdes.
datos_2015 = os.path.join(folder, 'Delitos_2015')
datos_2016 = os.path.join(folder, 'Delitos_2016')
datos_2017 = os.path.join(folder, 'Delitos_2017')
datos_2018 = os.path.join(folder, 'Delitos_2018')
datos_2019 = os.path.join(folder, 'Delitos_2019')
datos_2020 = os.path.join(folder, 'Delitos_2020')
datos_2021 = os.path.join(folder, 'Delitos_2021')
datos_2022 = os.path.join(folder, 'Delitos_2022')
datos_2023 = os.path.join(folder, 'Delitos_2023')
datos_2024 = os.path.join(folder, 'Delitos_2024')
distritos_shp = os.path.join(folder, 'Distritos_shape', 'DISTRITOS.shp')
poblacion = os.path.join(folder, 'Poblacion', 'poblacion_1_enero.csv')
Zonas_verdes_rast = os.path.join(folder, 'Zonas_verdes_rast', 'Zonas_verdes_30m.tif')

Una vez localizadas y seleccionadas las carpetas que contiene los conjuntos de datos con los que vamos a trabajar, comenzamos a modificar estos conjuntos de datos. Para ello, el primer paso es seleccionar únicamente las hojas dentro de los archivos excel que nos interesan. Después, creamos para cada archivo excel una columna que incluira la fecha del archivo, extrayendola a partir del nombre del mismo. Por último, concatenamos las diferentes hojas y archivos para obtener una base de datos con la que poder trabajar.

In [ ]:
# Lista de las hojas a leer de los arcchivos excel
hojas_a_leer = ['SEGURIDAD', 'DETENIDOS X DISTRITOS', 'ACCIDENTES', 'VENTA AMBULANTE', 'CONSUMO ALCOHOL', 'LOCALES']

# Función para obtener los archivos Excel en una carpeta
def obtener_archivos_excel(carpeta):
    return [f for f in os.listdir(carpeta) if f.endswith('.xlsx') or f.endswith('.xls')]

# Obtener los archivos Excel de las carpetas datos_2015 y datos_2024
archivos_2015 = obtener_archivos_excel(datos_2015)
archivos_2016 = obtener_archivos_excel(datos_2016)
archivos_2017 = obtener_archivos_excel(datos_2017)
archivos_2018 = obtener_archivos_excel(datos_2018)
archivos_2019 = obtener_archivos_excel(datos_2019)
archivos_2020 = obtener_archivos_excel(datos_2020)
archivos_2021 = obtener_archivos_excel(datos_2021)
archivos_2022 = obtener_archivos_excel(datos_2022)
archivos_2023 = obtener_archivos_excel(datos_2023)
archivos_2024 = obtener_archivos_excel(datos_2024)

# Lista para almacenar los DataFrames de las hojas seleccionadas
todos_los_datos = []

# Función para procesar los archivos de una carpeta y concatenar las hojas seleccionadas por columnas
def procesar_archivos(carpeta, archivos):
    for archivo in archivos:
        # Leer todas las hojas del archivo Excel
        xls = pd.ExcelFile(os.path.join(carpeta, archivo))

        # Lista temporal para almacenar las hojas que se van a concatenar por columnas
        hojas_temporales = []

        # Obtener el nombre correcto del archivo (sin el sufijo)
        nombre_archivo_correcto = archivo.split('.')[0]

        # Variable para controlar si se está en la primera hoja
        es_primera_hoja = True

        # Iterar sobre las hojas del archivo y leer solo las que están en la lista `hojas_a_leer`
        for nombre_hoja in xls.sheet_names:
            if nombre_hoja in hojas_a_leer:
                # Leer la hoja en un DataFrame, omitiendo las dos primeras filas
                df = pd.read_excel(xls, sheet_name=nombre_hoja, skiprows=2)

                # Si es la primera hoja, añadir la columna 'Fecha' y 'Distritos' con el nombre del archivo
                if es_primera_hoja:
                    df['Fecha'] = nombre_archivo_correcto

                if not es_primera_hoja:
                    df = df.iloc[:, 1:]  # Eliminar la primera columna

                es_primera_hoja = False

                # Añadir el DataFrame de la hoja a la lista temporal
                hojas_temporales.append(df)

        # Si hay hojas seleccionadas para este archivo, concatenarlas por columnas
        if hojas_temporales:
            df_archivo = pd.concat(hojas_temporales, axis=1)
            # Añadir el DataFrame concatenado por columnas a la lista de todos los datos
            todos_los_datos.append(df_archivo)

# Procesar los archivos de las carpetas 2015 y 2024
procesar_archivos(datos_2015, archivos_2015)
procesar_archivos(datos_2016, archivos_2016)
procesar_archivos(datos_2017, archivos_2017)
procesar_archivos(datos_2018, archivos_2018)
procesar_archivos(datos_2019, archivos_2019)
procesar_archivos(datos_2020, archivos_2020)
procesar_archivos(datos_2021, archivos_2021)
procesar_archivos(datos_2022, archivos_2022)
procesar_archivos(datos_2023, archivos_2023)
procesar_archivos(datos_2024, archivos_2024)

# Concatenar todos los DataFrames de diferentes archivos Excel por filas
df_final = pd.concat(todos_los_datos, axis=0, ignore_index=True)

Después de esto, hacemos una primera exploración de los datos, para comprobar que se han unido correctamente las distintas hojas y excels.

In [ ]:
df_final.head()

In [ ]:
df_final.info()

Algunas variables contienen la misma información pero han recibido nombres diferentes. Por ello, lo primero que tenemos que hacer es concatenar los datos del mismo tipo.

In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['CON HERIDOS', 'CON VICTIMAS']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['CON HERIDOS'] = nueva_columna


In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['SIN HERIDOS', 'SIN VICTIMAS']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['SIN HERIDOS'] = nueva_columna


In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['ADULTOS', 'MAYORES DE EDAD']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['ADULTOS'] = nueva_columna


In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['MENORES', 'MENORES DE 18 AÑOS']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['MENORES'] = nueva_columna


A continuación asignamos una variable para el shapefile que contiene las geometrías de los distritos. Después exploramos este shapefile para ver a partir de que columnas podemos unir los datos.

In [ ]:
distritos = gpd.read_file(distritos_shp)

Después exploramos el shapefile para ver que información contiene y saber a partir de que columna podemos unir la base de datos y el shapefile.

In [ ]:
print(distritos.head())

In [ ]:
print(distritos.columns)

Al comparar los datos del shapefile y la base de datos vemos que hay columnas que comparten información pero no se llaman igual, por lo que cambiamos el nombre de una del shapefile para realizar la unión.

In [ ]:
# Renombrar la columna 'shapefile_ID' a 'df_ID' para que coincidan
distritos = distritos.rename(columns={'DISTRI_MT': 'DISTRITOS'})

Nos aseguramos que ambas columnas están en el mismo formato para realizar la unión.

In [ ]:
df_final['DISTRITOS'] = df_final['DISTRITOS'].astype(str)
distritos['DISTRITOS'] = distritos['DISTRITOS'].astype(str)

Realizamos la unión a partir de la columna que comparte información.

In [ ]:
# Realizar la unión entre el DataFrame y el shapefile
df_union = df_final.merge(distritos[['DISTRITOS', 'geometry']], on='DISTRITOS', how='left')

Una vez realizada la unión, comenzamos a modificar la base de datos para poder trabajar con ella. En primer lugar, generamos una columna de año y otra de mes a partir de la columna generada a partir del nombre de los archivos.

In [ ]:
# Con este código separamos la columna fecha en dos nuevas columnas, año y mes.
df_union[['Mes', 'Año']] = df_union['Fecha'].str.split('_', expand=True)

# Mostrar las primeras filas del GeoDataFrame con las nuevas columnas
print(df_union[['Fecha', 'Mes', 'Año']])

Después generamos una columna con los meses como variable numérica. Tener la variable de meses de este modo puede llegar a ser útil de cara a algunas representaciones gráficas de los datos.

In [ ]:
# Diccionario que mapea los meses en palabras a su respectivo número
mes_a_numero = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

# Convertir la columna 'mes' de nombre a número
df_union['mes'] = df_union['Mes'].map(mes_a_numero)

A continuación, convertimos el data frame en un geodataframe.

In [ ]:
# Asegurarse de que es un GeoDataFrame
df_union = gpd.GeoDataFrame(df_union, geometry='geometry')

Después añadimos algunas columnas que podrían ser útiles a la hora de estudiar la base de datos, como el área de cada distrito.

In [ ]:
df_union['Area'] = df_union.geometry.area # Por defecto se calcula el área en metros cuadrados
df_union['Area_ha'] = df_union['Area'] / 10000 # Área en hectáreas
df_union['Area_km2'] = df_union['Area'] / 1000000 # Área en kilómetros cuadrados


Luego, eliminamos las columnas que no nos interesan de nuestra base de datos.

In [ ]:
#He puesto esto como ejemplo, pero se pueden poner otros o ninguno, depende de lo que acabemos haciendo.
df_union = df_union.drop(columns=['Fecha','DENUNCIAS','DETENIDOS E IMPUTADOS','DETENIDOS E INVESTIGADOS'])

Después de esto, incorporamos datos de población a nuestro dataframe. Como se puede ver a continuación, dan información sobre la población por distrito, lo cual podría ser una variable interesante a la hora de estudiar los delitos cometidos.

In [ ]:
# Leer el archivo CSV y seleccionar las primeras 22 columnas,
# que son las que contienen los datos que nos interesan
df_poblacion = pd.read_csv(poblacion, sep=";").head(21)

# Mostrar las primeras filas del DataFrame para verificar
df_poblacion.head()

In [ ]:
df_poblacion.info()

Después cambiamos el nombre de algunas columnas para que sea más facil trabajar con ellas en el futuro. Además, eliminamos las columnas que no nos interesan.

In [ ]:
# Cambiar nombres de algunas columnas para adecuarlos
df_poblacion = df_poblacion.rename(columns={'num_personas': 'poblacion'})
df_poblacion = df_poblacion.rename(columns={'num_personas_hombres': 'pob_hombres'})
df_poblacion = df_poblacion.rename(columns={'num_personas_mujeres': 'pob_mujeres'})
df_poblacion = df_poblacion.rename(columns={'distrito': 'DISTRITOS'})

# Eliminar columnas que no resultan interesantes en el analisis
df_poblacion = df_poblacion.drop(columns=['fecha', 'cod_municipio', 'municipio',
                                          'cod_barrio', 'barrio'])


Antes de poder unir estos datos con el resto necesitamos modificar el nombre de la columna de distritos para que coincidan.

In [ ]:
# Poner los nombres de los distritos en mayúsculas, también para manejar la unión
df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].str.upper()
df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].astype(str)

In [ ]:
# Eliminar espacios después de los guiones en los nombres de los distritos
# para mantener el mismo nombre y poder realizar la unión correctamente
df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].str.replace(r'\s*-\s*', '-', regex=True)
df_union['DISTRITOS'] = df_union['DISTRITOS'].str.replace(r'\s*-\s*', '-', regex=True)

df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].str.strip().str.upper().str.replace(r'\s*-\s*', '-', regex=True)
df_union['DISTRITOS'] = df_union['DISTRITOS'].str.strip().str.upper().str.replace(r'\s*-\s*', '-', regex=True)

Realizamos la unión y comprobamos que se ha realizado correctamente.

In [ ]:
df_union = df_union.merge(df_poblacion, on='DISTRITOS', how='left')

In [ ]:
df_union.head()

Después convertimos los datos de población a tipo numérico y calculamos la densidad de población (total, de hombres y de mujeres).

In [ ]:
# Convertir la columna 'Poblacion' a tipo numérico (por si tiene valores tipo str)
df_union['poblacion'] = pd.to_numeric(df_union['poblacion'], errors='coerce')
df_union['pob_hombres'] = pd.to_numeric(df_union['pob_hombres'], errors='coerce')
df_union['pob_mujeres'] = pd.to_numeric(df_union['pob_mujeres'], errors='coerce')

# Calcular densidad de población
df_union['densidad_pob_km2'] = df_union['poblacion'] / df_union['Area_km2']
df_union['densidad_pob_ha'] = df_union['poblacion'] / df_union['Area_ha']
df_union['densidad_pob_hombres'] = df_union['pob_hombres'] / df_union['Area_km2']
df_union['densidad_pob_mujeres'] = df_union['pob_mujeres'] / df_union['Area_km2']
df_union['densidad_pob_hombres_ha'] = df_union['pob_hombres'] / df_union['Area_ha']
df_union['densidad_pob_mujeres_ha'] = df_union['pob_mujeres'] / df_union['Area_ha']

Después pasamos a la limpieza de los datos. Para ello primero comprobamos si hay datos duplicados o valores nulos en nuestro dataframe.

In [ ]:
#Este código muestra las filas duplicadas.
df_union[df_union.duplicated(keep='first')]

In [ ]:
#Esta función nos permite saber si hay datos nulos
df_union.isnull().sum()

Como en las bases de datos originales había información sobre el total de la zona y sabiendo que estos datos no poseen geometrías y tampoco datos de población, aparecen como nulos. Por ello los eliminaremos.

In [ ]:
df_union = df_union.dropna()

Después hacemos una primera representación gráfica, en la que podemos observar como los datos que vamos a estudiar se ubican en los distintos distritos de la ciudad de Madrid.

In [ ]:
# Crear el mapa base sin leyenda
ax = df_union.plot(figsize=(10, 10), edgecolor='black', column='DISTRITOS', cmap='Set3', legend=False)

# Añadir etiquetas con mayor desplazamiento y fuente más pequeña
for x, y, label in zip(df_union.geometry.centroid.x, df_union.geometry.centroid.y, df_union['DISTRITOS']):
    ax.annotate(text=label, xy=(x, y), xytext=(5, 5), textcoords="offset points", fontsize=6, ha='center')

# Mostrar el mapa
plt.show()

In [ ]:
df_union.head()

**Ráster zonas verdes**

Después de esto, comenzamos a trabajar con datos ráster. Estos datos contienen información sobre las zonas verdes (jardines, matorral y arbolado) de los distritos de la ciudad de Madrid.

In [ ]:
zonas_verdes_rast = os.path.join(folder, 'Zonas_verdes_rast', 'Zonas_verdes_30m.tif')

Lo primero que haremos con estos datos es explorar sus características, como el sistema de referencia o tamaño de píxel, y visualizamos el ráster.

In [ ]:
# Verificar si el archivo existe
if os.path.exists(zonas_verdes_rast):
    # Cargar el ráster con rasterio
    with rasterio.open(zonas_verdes_rast) as src:
        print("Ráster cargado correctamente")
        print("Nombre del archivo:", src.name)
        print("Dimensiones (ancho x alto):", src.width, "x", src.height)
        print("Número de bandas:", src.count)
        print("Sistema de coordenadas (SRC):", src.crs)
        print("Tamaño del píxel:", src.res)
else:
    print("El archivo no se encontró en la ruta especificada.")

In [ ]:
# Abrir el archivo raster usando rasterio
with rasterio.open(zonas_verdes_rast) as src:
    # Usar dask para cargar los datos del raster en un array de dask
    data = da.from_array(src.read(1), chunks=(1000, 1000))  # "chunks" define el tamaño de los bloques

# Visualizar el array dask
plt.imshow(data.compute(), cmap='gray')  # .compute() realiza el cálculo y obtiene los resultados
plt.colorbar()
plt.title("Raster con Dask")
plt.show()

A continuación mostramos como podríamos hacer lo mismo utilizando xarray. Esta manera de hacerlo sería adecuada en jupyter notebook, pero en google collab da problemas.

In [ ]:
## Verificar si el archivo existe
#if os.path.exists(zonas_verdes_rast):
    ## Abrir el ráster con xarray y rasterio
    #zonas_verdes = xr.open_dataarray(zonas_verdes_rast, engine="rasterio")

    ## Mostrar información básica
    #print(zonas_verdes)

    ## Visualizar el ráster
    #zonas_verdes.plot(cmap="Greens")
    #plt.title("Visualización del ráster de zonas verdes con xarray")
    #plt.show()
#else:
    #print("El archivo no se encontró en la ruta especificada.")

Después de esto calculamos la superficie de zonas verdes por cada distrito de Madrid y verificamos que sean congruentes con las áreas de los distritos calculadas previamente.

In [ ]:
# Abrir el ráster
with rasterio.open(zonas_verdes_rast) as src:
    # Obtener el CRS del ráster
    raster_crs = src.crs
    transform = src.transform
    raster_array = src.read(1)  # Leer la primera banda
    pixel_area = (src.res[0] * src.res[1]) / 10_000  # Convertir de m² a ha

    # Guardar el CRS original de df_union
    original_crs = df_union.crs

    # Verificar y reproyectar df_union al CRS del ráster si es necesario
    if df_union.crs != raster_crs:
        df_union = df_union.to_crs(raster_crs)  # Reproyectar al CRS del ráster

    superficies = []

    for idx, barrio in df_union.iterrows():
        # Crear una máscara del barrio sobre el ráster
        mask = geometry_mask([barrio.geometry], transform=transform, invert=True, out_shape=raster_array.shape)

        # Extraer solo los píxeles dentro del barrio que contengan zonas verdes
        superficie_verde = np.sum(mask * (raster_array < 12)) * pixel_area

        superficies.append(superficie_verde)

    # Restaurar el CRS original de df_union
    df_union = df_union.to_crs(original_crs)

# Agregar la columna al GeoDataFrame
df_union["superficie_zonas_verdes_ha"] = superficies

# Mostrar los primeros valores para verificar
df_union[["DISTRITOS", "superficie_zonas_verdes_ha", "Area_ha"]].head()

Después representamos los distritos en función del área de zonas verdes que poseen.

In [ ]:
# Configurar el tamaño del mapa
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Graficar los barrios con clasificación por cuantiles
df_union.plot(column="superficie_zonas_verdes_ha", cmap="YlGn",
              linewidth=0.5, edgecolor="black", legend=True,
              scheme="quantiles", k=5, ax=ax)  # 5 clases por cuantiles

# Añadir títulos y etiquetas
ax.set_title("Superficie de Zonas Verdes por distrito (ha)", fontsize=14)
ax.set_axis_off()  # Ocultar ejes

# Mostrar el mapa
plt.show()

Realizamos esta misma representación con un mapa interactivo.

In [ ]:
# Guardar el EPSG original antes de cambiarlo
original_epsg = df_union.crs.to_string()

# Convertir el GeoDataFrame a EPSG:4326 (necesario para Folium)
df_union = df_union.to_crs(epsg=4326)

# Crear un mapa centrado en Madrid
m = folium.Map(location=[40.4168, -3.7038], zoom_start=11)

# Añadir capa de fondo de Google Maps (satélite)
folium.TileLayer(tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
                 attr="Google", name="Google Satellite").add_to(m)

# Crear un colormap para la superficie de zonas verdes
min_val = df_union["superficie_zonas_verdes_ha"].min()
max_val = df_union["superficie_zonas_verdes_ha"].max()
colormap = cm.linear.YlGn_09.scale(min_val, max_val)
colormap.caption = "Superficie de Zonas Verdes (ha)"
colormap.add_to(m)

# Función para asignar estilo con transparencia
def style_function(feature):
    value = feature["properties"]["superficie_zonas_verdes_ha"]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.05,  # Ajustar transparencia
    }

# Añadir los distritos como GeoJSON con el estilo definido
folium.GeoJson(
    df_union,
    name="Superficie de Zonas Verdes",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["DISTRITOS", "superficie_zonas_verdes_ha"],
                                  aliases=["Distrito:", "Zonas Verdes (ha):"],
                                  localize=True,
                                  labels=True,
                                  sticky=True)
).add_to(m)

# Añadir control de capas
folium.LayerControl().add_to(m)

# Mostrar el mapa directamente en Colab
display(m)

# Recuperar el EPSG original
df_union = df_union.to_crs(original_epsg)

Ahora en lugar de representarlo en función del área de zonas verdes hacemos una representación de la proporción de zonas verdes dentro de cada distrito.

In [ ]:
# Asegurar que el GeoDataFrame está en un sistema de coordenadas proyectadas para calcular correctamente su área
df_union = df_union.to_crs(epsg=3035)

# Calcular la superficie total del barrio en hectáreas (ahora en metros cuadrados porque estamos usando un CRS proyectado)
df_union["superficie_total_ha"] = df_union.geometry.area / 10_000  # m² a ha

# Reproyectar de nuevo a EPSG:4326 si es necesario para la visualización
df_union = df_union.to_crs(epsg=4326)

# Calcular la proporción de zonas verdes como porcentaje
df_union["proporcion_zonas_verdes"] = (df_union["superficie_zonas_verdes_ha"] / df_union["superficie_total_ha"]) * 100

# Crear un mapa centrado en Madrid
m = folium.Map(location=[40.4168, -3.7038], zoom_start=11)

# Añadir capa de fondo de Google Maps (satélite)
folium.TileLayer(tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
                 attr="Google", name="Google Satellite").add_to(m)

# Crear un colormap para la proporción de zonas verdes
min_val = df_union["proporcion_zonas_verdes"].min()
max_val = df_union["proporcion_zonas_verdes"].max()
colormap = cm.linear.YlGn_09.scale(min_val, max_val)
colormap.caption = "Proporción de Zonas Verdes (%)"
colormap.add_to(m)

# Función para asignar estilo con transparencia
def style_function(feature):
    value = feature["properties"]["proporcion_zonas_verdes"]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.05,  # Ajustar transparencia
    }

# Añadir los barrios como GeoJSON con el estilo definido
folium.GeoJson(
    df_union,
    name="Proporción de Zonas Verdes",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["DISTRITOS", "proporcion_zonas_verdes"],
                                  aliases=["Distrito:", "Proporción Zonas Verdes (%)"],
                                  localize=True,
                                  labels=True,
                                  sticky=False)
).add_to(m)

# Añadir control de capas
folium.LayerControl().add_to(m)

# Mostrar el mapa directamente en Colab
display(m)

# Recuperar el EPSG original
df_union = df_union.to_crs(original_epsg)

Por último, también hacemos una representación de los distritos por densidad de habitantes.

In [ ]:
# Convertir el GeoDataFrame a EPSG:4326 (necesario para Folium)
df_union = df_union.to_crs(epsg=4326)

# Crear un mapa centrado en Madrid
m = folium.Map(location=[40.4168, -3.7038], zoom_start=11)

# Añadir capa de fondo de Google Maps (satélite)
folium.TileLayer(tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
                 attr="Google", name="Google Satellite").add_to(m)

# Crear un colormap para la densidad de población
min_val = df_union["densidad_pob_km2"].min()
max_val = df_union["densidad_pob_km2"].max()
colormap = cm.linear.YlOrRd_09.scale(df_union["densidad_pob_km2"].min(), df_union["densidad_pob_km2"].max())
colormap.caption = "Densidad de Población (hab/km²)"
colormap.add_to(m)

# Función para asignar estilo con transparencia
def style_function(feature):
    value = feature["properties"]["densidad_pob_km2"]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.01,  # Ajustar transparencia
    }

# Añadir los barrios como GeoJSON con el estilo definido y formatear los valores de densidad de población a 2 decimales
folium.GeoJson(
    df_union,
    name="Densidad de población en los distritos de Madrid",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["DISTRITOS", "densidad_pob_km2", "poblacion"],
                                  aliases=["Distrito:", "Densidad de Población (hab/km²):", "Población:"],
                                  localize=True,
                                  labels=True,
                                  sticky=True)
).add_to(m)

# Añadir control de capas
folium.LayerControl().add_to(m)

# Mostrar el mapa directamente en Colab
display(m)

# Recuperar el EPSG original
df_union = df_union.to_crs(original_epsg)

## **4. Resultados**

Una vez explorados y preparados los datos comenzamos a generar resultados. En primer lugar, generamos histogramas y otro tipo de gráficospara ver la distribución de delitos cometidos. En el siguiente gráfico podemos ver la evolución de los tipos de delitos durante la última década.

In [ ]:
# Filtramos el DataFrame para mantener solo las columnas de interés
df_filtered = df_union[['Fecha'] + delitos]

# Número de subgráficos a crear (uno por cada tipo de delito)
n = len(delitos)

# Crear una figura con múltiples subgráficos (ajustamos el tamaño de la figura dependiendo de cuántos gráficos hay)
fig, axes = plt.subplots(n, 1, figsize=(10, 6 * n))

# Si solo hay un gráfico, axes no será un array, así que nos aseguramos de tratarlo como tal
if n == 1:
    axes = [axes]

# Iteramos sobre los delitos y creamos los gráficos
for i, column in enumerate(delitos):
    # Agrupamos por fecha y sumamos los delitos de ese tipo
    daily_crimes = df_filtered.groupby('Fecha')[column].sum()

    # Graficamos
    axes[i].plot(daily_crimes.index, daily_crimes, label=column, color='tab:blue')
    axes[i].set_title(f"Evolución de {column.lower()} por fecha")
    axes[i].set_xlabel('Fecha')
    axes[i].set_ylabel(f'Número de {column.lower()}')
    axes[i].tick_params(axis='x', rotation=45)  # Rotamos las etiquetas de la fecha
    axes[i].legend()
    axes[i].grid(True)

# Ajustamos el layout para evitar solapamiento
plt.tight_layout()
plt.show()

Después comprobamos en qué lugar y fecha se han cometido más delitos de cada tipo.

In [ ]:
# Filtramos las filas donde 'DISTRITOS' no es NaN
df_filtered = df_union[df_union['DISTRITOS'].notna()]

# Lista de columnas relacionadas con delitos que queremos procesar
delitos = [
    'RELACIONADAS CON LAS PERSONAS',
    'RELACIONADAS CON EL PATRIMONIO',
    'POR TENENCIA DE ARMAS',
    'POR TENENCIA DE DROGAS',
    'POR CONSUMO DE DROGAS',
    'PROPIEDAD INTELECTUAL E INDUSTRIAL',
    'INFRACCIONES ALIMENTARIAS',
    'INSPECCIONES Y ACTUACIONES',
    'CON HERIDOS',
    'SIN HERIDOS',
    'ADULTOS',
    'MENORES'
]

# Iteramos solo sobre las columnas de delitos
for columna in delitos:
    if columna in df_filtered.columns and pd.api.types.is_numeric_dtype(df_filtered[columna]):
        max_index = df_filtered[columna].idxmax()  # Obtenemos el índice del valor máximo
        max_distrito = df_filtered.loc[max_index, 'DISTRITOS']  # Distrito correspondiente
        max_fecha = df_filtered.loc[max_index, 'Fecha']  # Fecha correspondiente
        print(f"Distrito con más delitos en '{columna.lower()}': {max_distrito} en la fecha {max_fecha}")


Ahora generamos un gráfico interactivo que permita al usuario introducir el tipo de delito y fecha que desee visualizar, utilizando desplegables.

In [ ]:
# Nos aseguramos de que las columnas de 'Año' y 'mes' son de tipo string.
df_union['Año'] = df_union['Año'].astype(str)
df_union['mes'] = df_union['mes'].astype(str)

# Crear una nueva columna combinada 'Año-Mes' para la animación.
df_union['Año-mes'] = df_union['Año'] + '-' + df_union['mes']

# Convertir la columna 'Año-Mes' a tipo fecha para ordenar cronológicamente.
df_union['Fecha'] = pd.to_datetime(df_union['Año-mes'], format='%Y-%m')

# Eliminar la columna 'geometry' antes de hacer cualquier operación de agrupamiento ya que da problemas.
df_union_sin_geometry = df_union.drop(columns=['geometry',  'Mes', 'mes', 'Area',
                                               'Area_ha', 'Area_km2', 'cod_distrito', 'poblacion',
                                               'pob_hombres', 'pob_mujeres', 'densidad_pob_km2',
                                               'densidad_pob_ha', 'densidad_pob_hombres',
                                               'densidad_pob_mujeres', 'densidad_pob_hombres_ha',
                                               'densidad_pob_mujeres_ha'])

# Definir las opciones para el desplegable de tipo de delito.
opciones_tipo_delito = [
    'RELACIONADAS CON LAS PERSONAS',
    'RELACIONADAS CON EL PATRIMONIO',
    'POR TENENCIA DE ARMAS',
    'POR TENENCIA DE DROGAS',
    'POR CONSUMO DE DROGAS',
    'PROPIEDAD INTELECTUAL E INDUSTRIAL',
    'INFRACCIONES ALIMENTARIAS',
    'INSPECCIONES Y ACTUACIONES',
    'CON HERIDOS',
    'SIN HERIDOS',
    'ADULTOS',
    'MENORES'
]

# Crear el widget desplegable para seleccionar el tipo de delito.
dropdown_tipo_delito = widgets.Dropdown(
    options=opciones_tipo_delito,
    description='Tipo de delito:',
    value='MENORES',  # Valor por defecto
    disabled=False
)

# Crear el widget de opción para seleccionar cómo agrupar los datos (por fecha, distrito o año)
dropdown_agrupacion = widgets.Dropdown(
    options=['Fecha', 'DISTRITOS', 'AÑO'],
    description='Agrupar por:',
    value='Fecha',  # Valor por defecto
    disabled=False
)

# Crear el widget para seleccionar el distrito
dropdown_distrito = widgets.Dropdown(
    options=['Todos'] + list(df_union_sin_geometry['DISTRITOS'].unique()),
    description='Distrito:',
    value='Todos',  # Valor por defecto 'Todos'
    disabled=False
)


# Función para actualizar el gráfico según la selección del usuario
def actualizar_grafico(tipo_delito_seleccionado, agrupacion_seleccionada, distrito_seleccionado):
    # Si se ha seleccionado un distrito específico, filtrar los datos por ese distrito
    if distrito_seleccionado != 'Todos':
        df_filtrado = df_union_sin_geometry[df_union_sin_geometry['DISTRITOS'] == distrito_seleccionado]
    else:
        df_filtrado = df_union_sin_geometry  # Si se selecciona 'Todos', no filtrar por distrito

    # Seleccionar solo las columnas numéricas para realizar la agregación
    columnas_numericas = df_filtrado.select_dtypes(include=['number']).columns

    # Agrupar los datos por la opción seleccionada, asegurándose de no incluir la columna 'geometry'
    if agrupacion_seleccionada == 'Fecha':
        # Agrupar por fecha
        datos_agrupados = df_filtrado.groupby('Fecha')[columnas_numericas].sum()
    elif agrupacion_seleccionada == 'DISTRITOS':
        # Agrupar por distrito
        datos_agrupados = df_filtrado.groupby('DISTRITOS')[columnas_numericas].sum()
    elif agrupacion_seleccionada == 'AÑO':
        # Agrupar por Año
        datos_agrupados = df_filtrado.groupby('Año')[columnas_numericas].sum()

    # Seleccionar la columna de tipo de delito seleccionada
    tipo_delito = datos_agrupados[tipo_delito_seleccionado]

    # Graficar el total de delitos por tipo en el tiempo o distrito
    plt.figure(figsize=(12, 8))

    # Graficar los delitos por fecha, distrito o año, dependiendo de la selección
    tipo_delito.plot(kind='bar', stacked=True, figsize=(12, 8))

    # Personalizar la gráfica
    plt.title(f'Número total de delitos de tipo "{tipo_delito_seleccionado}" agrupados por {agrupacion_seleccionada}')
    plt.xlabel(agrupacion_seleccionada)
    plt.ylabel('Cantidad de delitos')

    # Ajustar las etiquetas del eje X dependiendo de la agrupación
    if agrupacion_seleccionada == 'Fecha':
        # Obtener las posiciones de los ticks en el eje X
        ticks_pos = range(len(tipo_delito))

        # Mostrar solo 1 de cada 6 etiquetas en el eje X porque no entran bien
        plt.xticks(ticks_pos[::6], tipo_delito.index.strftime('%Y-%m')[::6], rotation=90, fontsize=7)
    else:
        # Para otros casos (DISTRITOS o AÑO), no cambiar las etiquetas
        plt.xticks(rotation=90, fontsize=7)

    plt.tight_layout()
    plt.legend(title='Tipo de delito')
    plt.show()

# Mostrar los widgets y vincularlos a la función de actualización
widgets.interactive(actualizar_grafico,
                    tipo_delito_seleccionado=dropdown_tipo_delito,
                    agrupacion_seleccionada=dropdown_agrupacion,
                    distrito_seleccionado=dropdown_distrito)

A continuación creamos un mapa de calor para ver como se relacionan entre sí las distintas variables con las que contamos en nuestra base de datos. Si nos centramos en la relación entre los distintos tipos de delitos y otras variables podemos ver que la presencia de algunos tipos de delitos presenta una correlación positiva con la presencia de otros tipos de delitos. Algunos de estos ejemplos tienen bastante lógica, como la relación positiva que se da entre delitos por tenencia de drogas y delitos por consumo de drogas. También parece existir una correlación positiva entre varios tipos de delitos y la densidad de habitantes. En cambio, no parece haber ningún tipo de relación entre la proporción de superficies verdes y los delitos cometidos.

In [ ]:
# Crear mapa de calor para los tipos de delitos
plt.figure(figsize = (14,8))
sns.heatmap(df_union[[ 'RELACIONADAS CON LAS PERSONAS',
       'RELACIONADAS CON EL PATRIMONIO', 'POR TENENCIA DE ARMAS',
       'POR TENENCIA DE DROGAS', 'POR CONSUMO DE DROGAS',
       'PROPIEDAD INTELECTUAL E INDUSTRIAL', 'INFRACCIONES ALIMENTARIAS',
       'INSPECCIONES Y ACTUACIONES', 'CON HERIDOS', 'SIN HERIDOS', 'ADULTOS',
       'MENORES',   'Area',  'densidad_pob_ha',   'superficie_zonas_verdes_ha',
        'proporcion_zonas_verdes']].corr(),annot=True,square=True,
            cmap='RdBu',
            vmax=1,
            vmin=-1)
plt.title('Correlación entre variables',size=18);
plt.xticks(size=12)
plt.yticks(size=12)
plt.show()

Con el siguiente gráfico de bigotes podemos observar datos como la media de un determinado delito para una fecha y lugar en concreto. También podemos ver como muchos de los datos utilizados son interpretados como outlayers, lo cual es importante si en un fututo quisieramos realizar un estudio estadístico más avanzado.


In [ ]:
# Crear el widget desplegable para seleccionar el tipo de delito
dropdown_tipo_delito = widgets.Dropdown(
    options=opciones_tipo_delito,
    description='Tipo de delito:',
    value='MENORES',  # Valor por defecto
    disabled=False
)

# Crear el widget de opción para seleccionar cómo agrupar los datos (por fecha, distrito o año)
dropdown_agrupacion = widgets.Dropdown(
    options=['DISTRITOS', 'AÑO'],
    description='Agrupar por:',
    value='AÑO',  # Valor por defecto
    disabled=False
)

# Crear el widget para seleccionar el distrito
dropdown_distrito = widgets.Dropdown(
    options=['Todos'] + list(df_union_sin_geometry['DISTRITOS'].unique()),
    description='Distrito:',
    value='Todos',  # Valor por defecto 'Todos'
    disabled=False
)

# Función para actualizar el gráfico de delitos
def actualizar_grafico(tipo_delito_seleccionado, agrupacion_seleccionada, distrito_seleccionado):
    # Si se ha seleccionado un distrito específico, filtrar los datos por ese distrito
    if distrito_seleccionado != 'Todos':
        df_filtrado = df_union_sin_geometry[df_union_sin_geometry['DISTRITOS'] == distrito_seleccionado]
    else:
        df_filtrado = df_union_sin_geometry  # Si se selecciona 'Todos', no filtrar por distrito

    # Seleccionar solo las columnas numéricas para realizar la agregación
    columnas_numericas = df_filtrado.select_dtypes(include=['number']).columns

    # Seleccionar la columna de tipo de delito
    tipo_delito = df_filtrado[tipo_delito_seleccionado]

    # Graficar el boxplot de los delitos por tipo (por distrito o año)
    plt.figure(figsize=(12, 8))

    # Boxplot sin agrupación, mostrando todos los registros
    if agrupacion_seleccionada == 'DISTRITOS':
        # Boxplot mostrando todos los registros por distrito
        df_filtrado.boxplot(column=tipo_delito_seleccionado, by='DISTRITOS', vert=False, patch_artist=True, figsize=(12, 8))
    elif agrupacion_seleccionada == 'AÑO':
        # Boxplot mostrando todos los registros por año
        df_filtrado.boxplot(column=tipo_delito_seleccionado, by='Año', vert=False, patch_artist=True, figsize=(12, 8))

    # Personalizar la gráfica
    plt.title(f'Distribución de delitos de tipo "{tipo_delito_seleccionado}"')
    plt.suptitle('')  # Eliminar título por defecto de 'by'
    plt.xlabel('Cantidad de delitos')
    plt.ylabel(agrupacion_seleccionada)

    # Ajustar los valores del eje X
    plt.xticks(rotation=90, fontsize=7)

    # Ajustar el tamaño de los valores en el eje Y (sin "Fecha")
    plt.yticks(fontsize=7)

    plt.tight_layout()
    plt.show()

# Mostrar los widgets y vincularlos a la función de actualización
widgets.interactive(actualizar_grafico,
                    tipo_delito_seleccionado=dropdown_tipo_delito,
                    agrupacion_seleccionada=dropdown_agrupacion,
                    distrito_seleccionado=dropdown_distrito)

A continuación generamos gráficos de dispersión para poder ver la distribución de los datos de estudio según la fecha, tipo de delito y distrito. El primero de los gráficos es con los datos agrupados por periodo de tiempo y el siguiente con los datos brutos.

In [ ]:
# Crear el widget desplegable para seleccionar el tipo de delito
dropdown_tipo_delito = widgets.Dropdown(
    options=opciones_tipo_delito,
    description='Tipo de delito:',
    value='MENORES',  # Valor por defecto
    disabled=False
)

# Crear el widget de opción para seleccionar cómo agrupar los datos (por fecha, distrito o año)
dropdown_agrupacion = widgets.Dropdown(
    options=['Fecha', 'DISTRITOS', 'AÑO'],  # Agregar la opción 'AÑO'
    description='Agrupar por:',
    value='Fecha',  # Valor por defecto
    disabled=False
)

# Crear el widget para seleccionar el distrito
dropdown_distrito = widgets.Dropdown(
    options=['Todos'] + list(df_union_sin_geometry['DISTRITOS'].unique()),
    description='Distrito:',
    value='Todos',  # Valor por defecto 'Todos'
    disabled=False
)

# Función para actualizar el gráfico de dispersión
def actualizar_grafico(tipo_delito_seleccionado, agrupacion_seleccionada, distrito_seleccionado):
    # Si se ha seleccionado un distrito específico, filtrar los datos por ese distrito
    if distrito_seleccionado != 'Todos':
        df_filtrado = df_union_sin_geometry[df_union_sin_geometry['DISTRITOS'] == distrito_seleccionado]
    else:
        df_filtrado = df_union_sin_geometry  # Si se selecciona 'Todos', no filtrar por distrito

    # Seleccionar solo las columnas numéricas para realizar la agregación
    columnas_numericas = df_filtrado.select_dtypes(include=['number']).columns

    # Agrupar los datos por la opción seleccionada, asegurándose de no incluir la columna 'geometry'
    if agrupacion_seleccionada == 'Fecha':
        # Agrupar por fecha
        datos_agrupados = df_filtrado.groupby('Fecha')[columnas_numericas].sum()
    elif agrupacion_seleccionada == 'DISTRITOS':
        # Agrupar por distrito
        datos_agrupados = df_filtrado.groupby('DISTRITOS')[columnas_numericas].sum()
    elif agrupacion_seleccionada == 'AÑO':
        # Agrupar por Año
        datos_agrupados = df_filtrado.groupby('Año')[columnas_numericas].sum()

    # Seleccionar la columna de tipo de delito seleccionada
    tipo_delito = datos_agrupados[tipo_delito_seleccionado]

    # Graficar el gráfico de dispersión de los delitos por tipo (por fecha, distrito o año)
    plt.figure(figsize=(12, 8))

    # Gráfico de dispersión agrupado por fecha, distrito o año
    if agrupacion_seleccionada == 'Fecha':
        # Dispersión agrupada por fecha
        plt.scatter(datos_agrupados.index, tipo_delito, color='blue', alpha=0.6)
    elif agrupacion_seleccionada == 'DISTRITOS':
        # Dispersión agrupada por distrito
        plt.scatter(datos_agrupados.index, tipo_delito, color='green', alpha=0.6)
    elif agrupacion_seleccionada == 'AÑO':
        # Dispersión agrupada por año
        plt.scatter(datos_agrupados.index, tipo_delito, color='red', alpha=0.6)

    # Personalizar la gráfica
    plt.title(f'Distribución de delitos de tipo "{tipo_delito_seleccionado}" agrupados por {agrupacion_seleccionada}')
    plt.xlabel(agrupacion_seleccionada)
    plt.ylabel(f'Cantidad de delitos de tipo "{tipo_delito_seleccionado}"')

    # Ajustar el tamaño de los valores en el eje X
    plt.xticks(rotation=90, fontsize=7)

    plt.tight_layout()
    plt.show()

# Mostrar los widgets y vincularlos a la función de actualización
widgets.interactive(actualizar_grafico,
                    tipo_delito_seleccionado=dropdown_tipo_delito,
                    agrupacion_seleccionada=dropdown_agrupacion,
                    distrito_seleccionado=dropdown_distrito)

In [ ]:
# Crear el widget desplegable para seleccionar el tipo de delito
dropdown_tipo_delito = widgets.Dropdown(
    options=opciones_tipo_delito,
    description='Tipo de delito:',
    value='MENORES',  # Valor por defecto
    disabled=False
)

# Crear el widget de opción para seleccionar cómo agrupar los datos (por fecha, distrito o año)
dropdown_agrupacion = widgets.Dropdown(
    options=['Fecha', 'DISTRITOS', 'AÑO'],  # Agregar la opción 'AÑO'
    description='Agrupar por:',
    value='Fecha',  # Valor por defecto
    disabled=False
)

# Crear el widget para seleccionar el distrito
dropdown_distrito = widgets.Dropdown(
    options=['Todos'] + list(df_union_sin_geometry['DISTRITOS'].unique()),
    description='Distrito:',
    value='Todos',  # Valor por defecto 'Todos'
    disabled=False
)

# Función para actualizar el gráfico de dispersión
def actualizar_grafico(tipo_delito_seleccionado, agrupacion_seleccionada, distrito_seleccionado):
    # Si se ha seleccionado un distrito específico, filtrar los datos por ese distrito
    if distrito_seleccionado != 'Todos':
        df_filtrado = df_union_sin_geometry[df_union_sin_geometry['DISTRITOS'] == distrito_seleccionado]
    else:
        df_filtrado = df_union_sin_geometry  # Si se selecciona 'Todos', no filtrar por distrito

    # Seleccionar solo las columnas numéricas para realizar la agregación
    columnas_numericas = df_filtrado.select_dtypes(include=['number']).columns

    # Seleccionar los registros individuales (sin agrupar)
    tipo_delito = df_filtrado[tipo_delito_seleccionado]

    # Graficar el gráfico de dispersión de los delitos por tipo (sin agrupar por fecha, distrito o año)
    plt.figure(figsize=(12, 8))

    # Dispersión de los delitos individuales
    if agrupacion_seleccionada == 'Fecha':
        plt.scatter(df_filtrado['Fecha'], tipo_delito, color='blue', alpha=0.6, label="Fecha")
    elif agrupacion_seleccionada == 'DISTRITOS':
        plt.scatter(df_filtrado['DISTRITOS'], tipo_delito, color='green', alpha=0.6, label="Distrito")
    elif agrupacion_seleccionada == 'AÑO':
        plt.scatter(df_filtrado['Año'], tipo_delito, color='red', alpha=0.6, label="Año")

    # Personalizar la gráfica
    plt.title(f'Distribución de delitos de tipo "{tipo_delito_seleccionado}" sin agrupar por {agrupacion_seleccionada}')
    plt.xlabel(agrupacion_seleccionada)
    plt.ylabel(f'Cantidad de delitos de tipo "{tipo_delito_seleccionado}"')

    # Ajustar el tamaño de los valores en el eje X
    plt.xticks(rotation=90, fontsize=7)

    # Agregar leyenda
    plt.legend()

    plt.tight_layout()
    plt.show()

# Mostrar los widgets y vincularlos a la función de actualización
widgets.interactive(actualizar_grafico,
                    tipo_delito_seleccionado=dropdown_tipo_delito,
                    agrupacion_seleccionada=dropdown_agrupacion,
                    distrito_seleccionado=dropdown_distrito)


Después, utilizamos la función describe para mostrar algunas estadísticas interesantes en relación a cada tipo de delito.

In [ ]:
df_union[[ 'RELACIONADAS CON LAS PERSONAS',
       'RELACIONADAS CON EL PATRIMONIO', 'POR TENENCIA DE ARMAS',
       'POR TENENCIA DE DROGAS', 'POR CONSUMO DE DROGAS',
       'PROPIEDAD INTELECTUAL E INDUSTRIAL', 'INFRACCIONES ALIMENTARIAS',
       'INSPECCIONES Y ACTUACIONES', 'CON HERIDOS', 'SIN HERIDOS', 'ADULTOS',
       'MENORES']].describe()

También creamos una tabla en la que poder ver estas estadísticas en función del tipo de delito y el lugar en el que se cometen.

In [ ]:
# Crear el widget desplegable para seleccionar el tipo de delito
dropdown_tipo_delito = widgets.Dropdown(
    options=opciones_tipo_delito,
    description='Tipo de delito:',
    value='MENORES',  # Valor por defecto
    disabled=False
)

# Crear el desplegable para elegir el distrito
dropdown_distrito = widgets.Dropdown(
    options=['Todos'] + list(df_union_sin_geometry['DISTRITOS'].unique()),
    value='Todos',  # Valor por defecto
    description='Distrito:',
    disabled=False
)

# Función para calcular todas las estadísticas y mostrarlas en una tabla
def calcular_estadisticas(tipo_columna, distrito_seleccionado):
    if tipo_columna in df_union_sin_geometry.columns:
        # Filtrar los datos por distrito, si no es 'Todos'
        if distrito_seleccionado != 'Todos':
            df_filtrado = df_union_sin_geometry[df_union_sin_geometry['DISTRITOS'] == distrito_seleccionado]
        else:
            df_filtrado = df_union_sin_geometry  # Si 'Todos', no filtrar por distrito

        # Calcular todas las estadísticas de forma simultánea
        media = df_filtrado.groupby(['Año', 'DISTRITOS'])[tipo_columna].mean()
        mediana = df_filtrado.groupby(['Año', 'DISTRITOS'])[tipo_columna].median()
        desviacion_estandar = df_filtrado.groupby(['Año', 'DISTRITOS'])[tipo_columna].std()
        maximo = df_filtrado.groupby(['Año', 'DISTRITOS'])[tipo_columna].max()
        minimo = df_filtrado.groupby(['Año', 'DISTRITOS'])[tipo_columna].min()
        cuartiles = df_filtrado.groupby(['Año', 'DISTRITOS'])[tipo_columna].quantile([0.25, 0.5, 0.75])

        # Crear un DataFrame con todas las estadísticas
        stats_df = pd.DataFrame({
            'Media': media,
            'Mediana': mediana,
            'Desviación Estándar': desviacion_estandar,
            'Máximo': maximo,
            'Mínimo': minimo
        })

        # Convertir los cuartiles en un DataFrame
        cuartiles_df = cuartiles.unstack(level=-1).rename(columns={0.25: 'Q1', 0.5: 'Mediana', 0.75: 'Q3'})

        # Concatenar los cuartiles al DataFrame de estadísticas
        result_df = pd.concat([stats_df, cuartiles_df], axis=1)

        # Mostrar la tabla resultante
        display(result_df)
    else:
        print(f"El tipo de columna '{tipo_columna}' no es válido.")

# Conectar los widgets con la función
widgets.interactive(calcular_estadisticas, tipo_columna=dropdown_tipo_delito, distrito_seleccionado=dropdown_distrito)

También generamos pairplots, para ver la relación entre los distintos tipos de delitos. Debido a que la base de datos maneja varios tipos de delito, la visibilidad de los distintos gráficos no es muy buena pero a pesar de ello se puede apreciar como entre muchos de los delitos no existe relación y en los que si que existe la relación es positiva.

In [ ]:
# Crear el pairplot
sns.pairplot(df_union[[ 'RELACIONADAS CON LAS PERSONAS',
       'RELACIONADAS CON EL PATRIMONIO', 'POR TENENCIA DE ARMAS',
       'POR TENENCIA DE DROGAS', 'POR CONSUMO DE DROGAS',
       'PROPIEDAD INTELECTUAL E INDUSTRIAL', 'INFRACCIONES ALIMENTARIAS',
       'INSPECCIONES Y ACTUACIONES', 'CON HERIDOS', 'SIN HERIDOS', 'ADULTOS',
       'MENORES']],
             markers="+",
             diag_kind="kde",
             kind='reg',
             plot_kws={'line_kws': {'color': '#aec6cf'},
                       'scatter_kws': {'alpha': 0.7, 'color': 'red'}},
             corner=True)

También hemos generado un gráfico de sectores, que nos permite estudiar con facilidad que tipo de delito es el más habitual en cada distrito.

In [ ]:
# Crear el widget desplegable para seleccionar el tipo de delito
dropdown_tipo_delito = widgets.Dropdown(
    options=opciones_tipo_delito,
    description='Tipo de delito:',
    value='MENORES',  # Valor por defecto
    disabled=False
)

# Función para actualizar el gráfico de sectores
def actualizar_grafico(tipo_delito_seleccionado):
    # Filtrar los datos para contar los delitos por distrito
    tipo_delito = df_union_sin_geometry[tipo_delito_seleccionado]

    # Contar los delitos por distrito
    delitos_por_distrito = df_union_sin_geometry.groupby('DISTRITOS')[tipo_delito_seleccionado].sum()

    # Crear el gráfico de sectores
    plt.figure(figsize=(10, 8))
    wedges, texts, autotexts = plt.pie(delitos_por_distrito,
                                       autopct='%1.1f%%',
                                       startangle=90,
                                       colors=plt.cm.Paired.colors,
                                       textprops={'fontsize': 6})  # Ajustar el tamaño de la fuente para los porcentajes

    # Personalizar el gráfico
    plt.title(f'Distribución de delitos de tipo "{tipo_delito_seleccionado}" por distrito')
    plt.axis('equal')  # Aseguramos que el gráfico sea un círculo perfecto

    # Crear la leyenda
    plt.legend(wedges, delitos_por_distrito.index, title="Distritos", loc="center left", bbox_to_anchor=(1, 0.5))

    plt.tight_layout()
    plt.show()

# Mostrar los widgets y vincularlos a la función de actualización
widgets.interactive(actualizar_grafico, tipo_delito_seleccionado=dropdown_tipo_delito)

Después de esto, creamos gráficos en los que se muestre la tendencia a lo largo del tiempo de los delitos en cada distrito. Al igual que en gráficos anteriores el usuario puede seleccionar que tipo de delito quiere estudiar y en que zona o periodo de tiempo. Si observamos los resultados obtenidos podemos ver como en la mayoría de delitos o hay una tendencia negativa (disminución con el paso del tiempo) o no hay ninguna tendencia (los delitos se mantienen relativamente estables con el paso del tiempo). Para saber las causas de esta disminución habría que hacer un estudio más profundo que tenga en cuenta la fecha de implementación de legislación que controle estos delitos, la evolución de los efectivos de las fuerzas de seguridad, el estado socio-económico y otros muchos factores que pueden estar influyendo en estos resultados.

In [ ]:
# Obtener los distritos únicos
distritos = df_union['DISTRITOS'].unique()

# Función para graficar los delitos agrupados por año o mes, y distrito con regresión lineal
def graficar_delito(periodo, delito, lugar):
    if lugar == 'Todos los lugares':
        if periodo == 'Año':
            # Agrupar por año (para todos los lugares)
            delitos_por_periodo = df_union.groupby('Año')[delito].sum()
        elif periodo == 'Mes':
            # Agrupar por mes (para todos los lugares)
            delitos_por_periodo = df_union.groupby('mes')[delito].sum()
            # Ordenar los meses numéricamente
            delitos_por_periodo = delitos_por_periodo.sort_index()  # Asegura que los meses estén ordenados
    else:
        if periodo == 'Año':
            # Agrupar por distrito y año (solo para el distrito seleccionado)
            delitos_por_periodo = df_union[df_union['DISTRITOS'] == lugar].groupby('Año')[delito].sum()
        elif periodo == 'Mes':
            # Agrupar por distrito y mes (solo para el distrito seleccionado)
            delitos_por_periodo = df_union[df_union['DISTRITOS'] == lugar].groupby('mes')[delito].sum()
            # Ordenar los meses numéricamente
            delitos_por_periodo = delitos_por_periodo.sort_index()  # Asegura que los meses estén ordenados

    # Aplicar regresión lineal
    X = np.array(delitos_por_periodo.index).reshape(-1, 1)  # Convertir el índice a un formato numérico
    y = delitos_por_periodo.values  # Valores de los delitos

    # Crear y ajustar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X, y)

    # Predecir los valores ajustados (línea de regresión)
    y_pred = model.predict(X)

    # Obtener la pendiente y el R cuadrado
    pendiente = model.coef_[0]  # Pendiente
    r_cuadrado = model.score(X, y)  # R cuadrado

    # Graficar los datos originales y la regresión lineal
    plt.figure(figsize=(10, 6))
    plt.plot(delitos_por_periodo.index, y, label='Datos originales', marker='o')
    plt.plot(delitos_por_periodo.index, y_pred, label='Tendencia (Regresión lineal)', linestyle='--', color='red')
    plt.title(f'Evolución de los delitos: {delito} por {periodo} en {lugar}')
    plt.xlabel(periodo)
    plt.ylabel('Número de delitos')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)

    # Si es por mes, cambiamos las etiquetas para los meses
    if periodo == 'Mes':
        plt.xticks(ticks=range(1, 13), labels=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                                                'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'])

    # Mostrar la pendiente y R cuadrado en el gráfico
    plt.text(0.3, 0.95, f'Pendiente: {pendiente:.4f}', transform=plt.gca().transAxes, fontsize=12, color='blue')
    plt.text(0.3, 0.90, f'R²: {r_cuadrado:.4f}', transform=plt.gca().transAxes, fontsize=12, color='blue')

    plt.show()

# Crear el widget para seleccionar el delito
dropdown_delito = widgets.Dropdown(
    options=delitos,
    value=delitos[0],  # Seleccionar el primer delito por defecto
    description='Delito:',
    disabled=False
)

# Crear el widget para seleccionar el periodo (Año o Mes)
dropdown_periodo = widgets.Dropdown(
    options=['Año', 'Mes'],
    value='Año',  # Seleccionar por defecto Año
    description='Período:',
    disabled=False
)

# Crear el widget para seleccionar el lugar (Todos los lugares o uno específico)
dropdown_lugar = widgets.Dropdown(
    options=['Todos los lugares'] + list(distritos),  # Agregar la opción de 'Todos los lugares'
    value='Todos los lugares',  # Seleccionar por defecto 'Todos los lugares'
    description='Lugar:',
    disabled=False
)

# Crear la interacción
interact(graficar_delito, periodo=dropdown_periodo, delito=dropdown_delito, lugar=dropdown_lugar)

Para facilitar el acceso a varios resultados de manera simultánea a continuación generamos una tabla con los distintos resultados de la pendiente y R cuadrado.

In [ ]:
# Desactivar los warnings
warnings.simplefilter('ignore')

# Obtener los distritos únicos
distritos = df_union['DISTRITOS'].unique()

# Crear un DataFrame vacío para almacenar los resultados
resultados = pd.DataFrame(columns=["Delito", "Distrito", "Período", "Pendiente", "R²"])

# Función para calcular la pendiente y el R cuadrado
def calcular_resultados(periodo, delito, distrito_seleccionado):
    global resultados

    if delito in df_union.columns:
        if distrito_seleccionado != 'Todos':
            # Filtrar por distrito
            df_filtrado = df_union[df_union['DISTRITOS'] == distrito_seleccionado]
        else:
            df_filtrado = df_union  # Si 'Todos', no filtrar por distrito

        if periodo == 'Año':
            # Agrupar por año (para todos los lugares o para un distrito)
            delitos_por_periodo = df_filtrado.groupby('Año')[delito].sum()
        elif periodo == 'Mes':
            # Agrupar por mes (para todos los lugares o para un distrito)
            delitos_por_periodo = df_filtrado.groupby('mes')[delito].sum()
            delitos_por_periodo = delitos_por_periodo.sort_index()  # Asegura que los meses estén ordenados

        # Aplicar regresión lineal
        X = np.array(delitos_por_periodo.index).reshape(-1, 1)  # Convertir el índice a un formato numérico
        y = delitos_por_periodo.values  # Valores de los delitos

        # Crear y ajustar el modelo de regresión lineal
        model = LinearRegression()
        model.fit(X, y)

        # Obtener la pendiente y el R cuadrado
        pendiente = model.coef_[0]  # Pendiente
        r_cuadrado = model.score(X, y)  # R cuadrado

        # Crear un DataFrame con el resultado actual
        resultado = pd.DataFrame({"Delito": [delito], "Distrito": [distrito_seleccionado], "Período": [periodo],
                                  "Pendiente": [pendiente], "R²": [r_cuadrado]})

        # Concatenar el nuevo resultado con el DataFrame de resultados
        global resultados
        resultados = pd.concat([resultados, resultado], ignore_index=True)

# Función para calcular todos los resultados de una vez
def calcular_todos_los_resultados():
    global resultados
    resultados = pd.DataFrame(columns=["Delito", "Distrito", "Período", "Pendiente", "R²"])

    # Calcular los resultados para todos los delitos, períodos y distritos
    for delito in delitos:
        for periodo in ['Año', 'Mes']:
            for distrito in ['Todos'] + list(distritos):
                calcular_resultados(periodo, delito, distrito)

    # Mostrar todos los resultados en la tabla
    display(resultados)

# Crear el botón para calcular todos los resultados de una vez
boton_calcular = widgets.Button(description="Calcular todos los resultados")
boton_calcular.on_click(lambda x: calcular_todos_los_resultados())

# Mostrar el botón en la interfaz
display(boton_calcular)


Para complementar estos resultados, también se ha estudiado la estacionalidad. La estacionalidad representa las variaciones en los datos que ocurren en intervalos regulares y que pueden estar debidos a patrones recorrentes o cíclicos. En los resultados podemos observar como para la mayoría de tipos de delitos existe una estacionalidad clara, es decir, se comenten más delitos de ese tipo en ciertas epocas del año. Por ejemplo, si estudiamos el consumo de drogas en el distrito de Latina podemos ver como cada años existen picos de delitos a mitad de año y final de año, indicando que hay algún factor influenciando estos resultados. Por ejemplo, podría estudiarse si esto está relacionado con el comienzo de periodos vacacionales o con algunas festividades en concreto.

Por otra parte, también se incluye una gráfica de residuos. Los residuos son los componentes de la serie temporal que no pueden explicarse por la tendencia o la estacionalidad, representando variaciones inesperadas o no predecibles en los datos. Como se puede observar en los resultados obtenidos, los residuos varían bastante entre tipo de delitos, indicando que alguno de ellos presentan patrones claramente predecibles con el tiempo y otros no.

In [ ]:
# Obtener los distritos sin repetir
distritos = df_union['DISTRITOS'].unique()

# Función para graficar los delitos agrupados por la columna Fecha con descomposición temporal
def graficar_delito(delito, lugar):
    if lugar == 'Todos los lugares':
        # Filtrar los datos según el delito seleccionado
        datos_delito = df_union[['Fecha', delito]]
    else:
        # Filtrar los datos según el distrito y delito seleccionado
        datos_delito = df_union[df_union['DISTRITOS'] == lugar][['Fecha', delito]]



    # Establecer la columna 'Fecha' como índice
    datos_delito.set_index('Fecha', inplace=True)

    # Resample para asegurarse de que los datos estén en un intervalo consistente (por ejemplo, mensual)
    datos_delito_resampled = datos_delito.resample('ME').sum()  # Resampling mensual

    # Descomposición temporal
    descomposicion = seasonal_decompose(datos_delito_resampled[delito], model='additive', period=12)  # Estacionalidad anual
    tendencia = descomposicion.trend
    estacionalidad = descomposicion.seasonal
    residuos = descomposicion.resid

    # Aplicar regresión lineal a los datos resampleados
    X = np.array(range(len(datos_delito_resampled))).reshape(-1, 1)  # Usamos el índice como variable temporal
    y = datos_delito_resampled[delito].values  # Valores de los delitos

    # Crear y ajustar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X, y)

    # Predecir los valores ajustados (línea de regresión)
    y_pred = model.predict(X)

    # Obtener la pendiente y el R cuadrado
    pendiente = model.coef_[0]  # Pendiente
    r_cuadrado = model.score(X, y)  # R cuadrado

    # Graficar los datos originales y la regresión lineal
    plt.figure(figsize=(12, 8))

    # Gráfico de la serie original y la tendencia
    plt.subplot(3, 1, 1)
    plt.plot(datos_delito_resampled.index, y, label='Datos originales', marker='o')
    plt.plot(datos_delito_resampled.index, tendencia, label='Tendencia', linestyle='--', color='red')
    plt.title(f'Evolución de los delitos: {delito} en {lugar}')
    plt.xlabel('Fecha')
    plt.ylabel('Número de delitos')
    plt.grid(True)
    plt.legend()

    # Estacionalidad
    plt.subplot(3, 1, 2)
    plt.plot(datos_delito_resampled.index, estacionalidad, label='Estacionalidad', color='green')
    plt.title(f'Estacionalidad de los delitos: {delito} en {lugar}')
    plt.xlabel('Fecha')
    plt.ylabel('Componente Estacional')
    plt.grid(True)
    plt.legend()

    # Residuos
    plt.subplot(3, 1, 3)
    plt.plot(datos_delito_resampled.index, residuos, label='Residuos', color='orange')
    plt.title(f'Residuos de la descomposición: {delito} en {lugar}')
    plt.xlabel('Fecha')
    plt.ylabel('Residuos')
    plt.grid(True)
    plt.legend()



    plt.tight_layout()
    plt.show()

# Crear el widget para seleccionar el delito
dropdown_delito = widgets.Dropdown(
    options=delitos,
    value=delitos[0],  # Seleccionar el primer delito por defecto
    description='Delito:',
    disabled=False
)

# Crear el widget para seleccionar el lugar (Todos los lugares o uno específico)
dropdown_lugar = widgets.Dropdown(
    options=['Todos los lugares'] + list(distritos),  # Agregar la opción de 'Todos los lugares'
    value='Todos los lugares',  # Seleccionar por defecto 'Todos los lugares'
    description='Lugar:',
    disabled=False
)

# Crear la interacción
interact(graficar_delito, delito=dropdown_delito, lugar=dropdown_lugar)


Todos estos resultados nos permiten dar respuesta a las preguntas planteadas al comienzo del EDA.

**1. ¿Cuáles son los distritos de Madrid con mayor número de delitos?**

Los distritos más céntricos, como el propio Centro o Salamanca, son en los que se cometen un mayor número de delitos.

**2.
¿Existe una correlación entre la superficie de zonas verdes y la cantidad de delitos en cada distrito?**

Los resultados nos indican que en base a los datos utilizados no existe una relación clara entre estas variables.

**3.
¿Las áreas con mayor densidad de población registran una mayor incidencia delictiva?**
La densidad de población es la variable más determinante en los delitos analizados, ya que las zonas con mayor densidad poblacional, como son los distritos del centro de la ciudad, tienen mayor concurrencia de hechos delictivos.

**4.
¿Cuál ha sido la evolución del número y tipo de delitos durante la última década?**

Existe una tendencia negativa (disminución de delitos) con el tiempo y hay un alto componente estacional para la mayoría de ellos lo que por un lado sugiere que las medidas que se hayan tomado para combatir estos delitos han sido efectivas y por otro lado, que existen factores externos que provocan repuntes de ciertos tipos de crímenes en determinadas épocas del año.




## **5. Evaluación crítica**

Antes de tomar los resultados obtenidos en este análisis exploratorio de datos como definitivos es importante tener en cuenta las debilidades y fortalezas del mismo.

Comenzando por la debilidades, la primera de ellas es que aunque se han incluido datos de población o zonas verdes para ver si están relacionados con el número de delitos, aún se podrían incluir muchas otras variables que ayuden al estudio, como la renta media por persona, nivel de estudios medios por persona o porcentaje de personas en el paro entre otros muchos ejemplos. Otra posible debilidad es que no hemos podido encontrar datos oficiales del mismo tipo que sean de fechas anteriores a las que hemos incluido, por lo que aunque el análisis temporal que hemos realizado no es malo, si que es cierto que no cubre una extensión temporal muy amplia y limita los resultados y conclusiones que podamos extraer del EDA. Otra posible debilidad es que no se han comparado los resultados obtenidos con los de otro EDA similar, ya que no hemos podido encontrar ninguno parecido. Esto nos habría permitido "validar" nuestros resultados, encontrar fallos o fuentes de error y mejorar nuestro EDA.

En cuanto a las fortalezas, los datos de partida son fiables ya que se han extraido de instituciones oficiales como el Ayuntamiento de Madrid. Además, existe continuidad en los datos, lo que nos permite estudiar la evolución de los delitos sin riesgo a que la falta de datos sesgue nuestros resultados. Otra fortaleza del estudio es que como los datos están divididos en función de distritos, nos permiten ver diferencias dentro de la ciudad de Madrid y dan pie a realizar un analisis espacial de los datos en el futuro. También es una fortaleza que los datos cubran todos los distritos de la ciudad, ya que esto significa que la cobertura espacial de la zona objetivo de estudio es total. Por último, una gran fortaleza del estudio es que se han podido responder, en mayor o menor medida, las preguntas que nos hemos planteado al inicio del estudio, permitiendonos cumplir nuestros objetivos iniciales.